In [7]:
!pip install cdqa

     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 102kB 6.4MB/s 
     |████████████████████████████████| 286kB 24.5MB/s 
     |████████████████████████████████| 10.5MB 49.3MB/s 
     |████████████████████████████████| 317kB 48.8MB/s 
     |████████████████████████████████| 6.7MB 43.8MB/s 
     |████████████████████████████████| 748.9MB 21kB/s 
     |████████████████████████████████| 92kB 13.0MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 890kB 42.9MB/s 
     |████████████████████████████████| 1.1MB 43.8MB/s 
     |████████████████████████████████| 102kB 13.5MB/s 
     |████████████████████████████████| 7.0MB 45.2MB/s 
     |████████████████████████████████| 71kB 11.2MB/s 
  Created wheel for cdqa: filename=cdqa-1.3.9-cp36-none-any.whl size=47640 sha256=bd445ae5442e1f00e06bd0e189f7d17d702403765a9b89bdcbe89387985f3b02
  Stored in directory: /root/.cache/pip/wheels/8b/9a/68/d3f7651ea29c30d1beb

In [1]:
!git clone https://github.com/jzhan1017/amazonqa.git

fatal: destination path 'amazonqa' already exists and is not an empty directory.


In [2]:
import json

train = []
for line in open('./amazonqa/datasets/train-qar_reduced.jsonl', 'r'):
    line = json.loads(line)
    train.append(line)

val = []
for line in open('./amazonqa/datasets/val-qar_reduced.jsonl', 'r'):
    line = json.loads(line)
    val.append(line)

test = []
for line in open('./amazonqa/datasets/test-qar_reduced.jsonl', 'r'):
    line = json.loads(line)
    test.append(line)

In [3]:
import pandas as pd

train_df = pd.DataFrame(train)
val_df = pd.DataFrame(val)
test_df = pd.DataFrame(test)

In [4]:
train_df = train_df[train_df['is_answerable']==0]
val_df = val_df[val_df['is_answerable']==0]
test_df = test_df[test_df['is_answerable']==0]

In [5]:
df_train = train_df[['questionText','review_snippets']]
df_train = df_train.rename(columns={"questionText": "title", "review_snippets": "paragraphs"})
df_val = val_df[['questionText','review_snippets']]
df_val = df_val.rename(columns={"questionText": "title", "review_snippets": "paragraphs"})
df_test = test_df[['questionText','review_snippets']]
df_test = df_test.rename(columns={"questionText": "title", "review_snippets": "paragraphs"})
df_all = pd.concat([df_train, df_val, df_test])

In [6]:
from cdqa.utils.download import download_model

download_model('distilbert-squad_1.1')

In [7]:
import pandas as pd
from ast import literal_eval
from cdqa.pipeline import QAPipeline

cdqa_pipeline = QAPipeline(reader='distilbert_qa.joblib')
cdqa_pipeline.fit_retriever(df=df_all)

/usr/local/lib/python3.6/dist-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)
100%|██████████| 231508/231508 [00:00<00:00, 16896913.54B/s]


QAPipeline(reader=BertQA(adam_epsilon=1e-08,
                         bert_model='distilbert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', ser...size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=0.85, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
           

In [8]:
from tqdm import tqdm

predictions = []
questions = df_all.title
for question in tqdm(questions):
  predictions.append(cdqa_pipeline.predict(query=question))

100%|██████████| 1546/1546 [12:53<00:00,  1.97it/s]


In [9]:
query = 'does it work with Win7?'
prediction = cdqa_pipeline.predict(query)
print('query: {}'.format(query))
print('answer: {}'.format(prediction[0]))
print('title: {}'.format(prediction[1]))
print('paragraph: {}'.format(prediction[2]))

query: does it work with Win7?
answer: This simply does not work
title: It will work fine with Android Box Minix Neo X8H and Ipad Mini 2?
paragraph: I bought this from ThinkGeek and made 2 mistakes. I spent $10 more than I should have, and I though I could find something better/cleaner than my Jerry rigged X-Box 360 controller. This simply does not work. I have been messing around for the past 4 hours trying every possible configuration and it just does not work as advertised.


In [10]:
result = pd.DataFrame()
result['question'] = train_df.questionText.append(val_df.questionText).append(test_df.questionText)
result['prediction'] = [i[0] for i in predictions]
result['answer'] = [[i['answerText'] for i in j] for j in train_df.answers] \
                    +[[i['answerText'] for i in j] for j in val_df.answers] \
                    +[[i['answerText'] for i in j] for j in test_df.answers]
result

,question,prediction,answer
0,Will this mouse work with Windows 8.1? I have ...,This game will not,"[Yes, it works without any problem... Tested f..."
5,Does it work on laptops windows 8,it does not work with windows 7,"[Yes it does, I run it in Windows 8 myself. It..."
6,This item as a combo is $157.99. They are sepa...,MX800,[Have the same question. Even Best Buy has th...
18,I am temporarily living in Europe. If I buy th...,i live in Europe. will this playstation work p...,"[Yes, in fact it's the only way to play on US ..."
25,Why is this game $67.99?? This game is 2.5 yea...,so it would work on win.7 thought it would be ...,[It's most likely because Nintendo is no longe...
26,Is the LED dimmable?,bigger than i expected,"[no, It can be turned OFF, but no, it can't be..."
30,Will this work with the Street Fighter Anniver...,I wouldn't recomend any other type of fight st...,"[Yes, Don't know for sure, might need a specia..."
31,will it work on gta v?,once u get n install the right driver,[I had the same question when I bought it but ...
38,MadCatz Saitek Cyborg RAT 7 Black/6400DPI? Hel...,empty,[The Rat 9 that i bought the other day just ca...
39,Pid and model number,3,[MN#: A-00060PN#: 881-000215I think these ar...


In [11]:
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

In [12]:
# evaluation sample
em_score = max((compute_exact_match(result.prediction[300], answer)) for answer in result.answer[300])
f1_score = max((compute_f1(result.prediction[300], answer)) for answer in result.answer[300])

print(f"Question: {train_df.questionText[300]}")
print(f"Prediction: {result.prediction[300]}")
print(f"True Answers: {result.answer[300]}")
print(f"EM: {em_score} \t F1: {f1_score}")

Question: A great game, especially the graphics. Does anybody else have blurry pictures in the strategy guide?
Prediction: cycling race
True Answers: ['No, my strategy guide does not have blurry pictures.', 'Its been a while since playing it but I had no trouble.  Great game.', "I don't remember using the strategy guide. Sorry!"]
EM: 0 	 F1: 0


In [1]:
# For colab users
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
result.to_csv('../content/drive/MyDrive/Colab Notebooks/result.csv')
#'/content/drive/MyDrive/output.csv'

In [2]:
import pandas as pd

result = pd.read_csv('/content/drive/MyDrive/output.csv', index_col='Unnamed: 0').reset_index(False)
result

,index,Baseline,Best Model,Human Answers,questions
0,0,"[""yes don't say the xbox on my of solitaire . ...","[""I would change my more on some good out afte...",['They are decent but you will not get surroun...,i know you should buy this with the mix amp fo...
1,1,['it .'],['3 off times controller'],"['3', ""The way it's made it will work for both...",Is it a 2 or 3 pin fan controller?
2,2,['I have bought the game it it get the game pa...,"[""i just got this and then to get the own me o...","[""Not it dosen't"", ""No, it doesn't. It is vali...",i wonder if the season pass got expired and ot...
3,3,"['yes can play the the on the , but you carefu...","[""sorry can at original it on 8 , but be very ...",['Steam activation codes and physical copies a...,If I don't have a disc drive and I get the dis...
4,4,['I no same problem as it it .'],['had the same button and go it . go to comput...,"['It does, read the manual it comes with on in...",Why doesnt the LA mod work?
...,...,...,...,...,...
627,629,['no balance wii wii controller 3DS is with th...,"['the original 7 system , which , in them , by...","[""Only when you're playing it. Once it's in tu...",Does the game help you tune your guitar?
628,630,"['yes , is']",['yes'],"['Yes they are.', 'Yes they are.... I was goin...",Are these updated to the latest firmware befor...
629,631,"['this is a . but mode , the multiplayer']","['it is both , 360 3 and online . . . . . ? fo...",['The Gamecube only supports AV cables. So if ...,Will this be able to hook up to an HDTV? Speci...
630,632,['it box is are are are are are are are are ar...,['hi'],"['Yes', 'Yes you can it works great', 'Yes, wi...",Can I play this game on windows 8.1?


In [ ]:
em_scores = []
f1_scores = []

for i in range(len(result)):
  answers = result['Human Answers']
  predictions = result['Best Model']
  em_scores.append(max((compute_exact_match(predictions[i], answer)) for answer in answers[i]))
  f1_scores.append(max((compute_f1(predictions[i], answer)) for answer in answers[i]))

print('Avg EM: {}'.format(sum(em_scores)/len(em_scores)))
print('Avg F1: {}'.format(sum(f1_scores)/len(f1_scores)))

Avg EM: 0.0
Avg F1: 0.047967673252670415


In [ ]:
result['Human Answers'][i]

'["Because, as a quick google search reveals, it\'s an xbox360 game released in 2007.", \'Technically James you are right that Lego Star Wars was originally for the original xbox, but they remade both the first and second games and combined them to get the game you see on this product.  Any other questions?\', "It\'s an XBOX 360 game. This is not a mistake.", \'Yeah - I bought it as an xbox 360 game and have played it on my 360\']'

In [9]:
import random

i = random.randint(0,len(result))

print('question: {}'.format(result['questions'][i]))
print('answer: {}'.format(result['Human Answers'][i]))
print('Baseline: {}'.format(result['Baseline'][i]))
print('prediction: {}'.format(result['Best Model'][i]))

question: We don't have connect but I was thinking about getting this game for my son for his birthday. Can he still play with a regular controller?
answer: ['Yes', 'Yes you can. Have fun!', 'yes you can play with a  regular controller.', 'My grandson plays it using a regular controller', 'Yes', 'Absolutely.']
Baseline: ['yes . I , have are are have are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are are']
prediction: ['absolutely .']
